In [2]:

import os
import pandas as pd
import numpy as np
import warnings
import datetime    
import psycopg2
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import font_manager, rc
import seaborn as sns
import math


font_path = "C:\\Windows\\Fonts\\NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
 

In [4]:
def get_dataframe_from_postgresql_table(table_name):
    import psycopg2
    
    conn = psycopg2.connect(
        host="localhost",
        port=5432,
        user="postgres",
        password="1234",
        database="ciel_imod"
    )


    cur = conn.cursor()

    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn) 
    return df
    

table_name = 'imod_original'
df = get_dataframe_from_postgresql_table(table_name)  # 1867722 rows × 52 columns
df.info()

C:\Users\ciel\AppData\Local\Temp\ipykernel_28340\2837179665.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2070737 entries, 0 to 2070736
Data columns (total 52 columns):
 #   Column       Dtype         
---  ------       -----         
 0   순번           int64         
 1   호출유형         object        
 2   지역           object        
 3   전화번호         object        
 4   배차차량         object        
 5   승무사원명        object        
 6   상태           object        
 7   요청일          datetime64[ns]
 8   취소일          datetime64[ns]
 9   승차정류장명       object        
 10  승차지시뮬레이션시간   datetime64[ns]
 11  승차지최초도착예정시간  datetime64[ns]
 12  승차지도착예정시간    datetime64[ns]
 13  승차지도착시간      datetime64[ns]
 14  승차시간         datetime64[ns]
 15  하차정류장명       object        
 16  하차지시뮬레이션시간   datetime64[ns]
 17  하차지최초도착예정시간  datetime64[ns]
 18  하차지도착예정시간    datetime64[ns]
 19  하차지도착시간      datetime64[ns]
 20  하차시간         datetime64[ns]
 21  eta산출api     object        
 22  eta최종갱신시간    object        
 23  탑승권아이디       float64       
 24  탑승권전달여부      object     

In [ ]:
def remove_columns(df, columns):
    '''리스트로 넘겨준 컬럼들 제거하는 함수'''
    df = df.drop(columns, axis=1)
    return df

def rename_columns(df, old_column_names, new_column_names):
    ''' 이름을 바꾸고 싶은 컬럼들을 리스트로 넘겨주면 바꿔주는 함수 '''
    df.rename(columns=dict(zip(old_column_names, new_column_names)), inplace=True)
    return df

def convert_to_datetime(df):
    '''모든 날짜 형식의 컬럼을 datetime 형식으로 변환하는 함수'''
    for col in ['요청시간','취소시간', '승차지시뮬레이션시간', '승차지최초도착예정시간','승차지도착예정시간', '승차지도착시간', '하차지시뮬레이션시간', '하차지최초도착예정시간','하차지도착예정시간','하차지도착시간','승차시간', '하차시간', '배차확정시간']:
        df[col] = pd.to_datetime(df[col])
    return df

def change_status(df):
    ''' 상태 컬럼의 값들을 탑승완료, 시스템취소로 변경하는 함수'''
    df.loc[df['상태'].str.startswith('탑승취소'), '상태'] = '탑승취소'
    df.loc[df['상태'].str.startswith('시스템취소'), '상태'] = '시스템취소'
    
    
    
    return df

def create_new_columns(df):
    '''데이터 분석을 위한 새로운 컬럼 추가하는 함수'''    
    
    df = df.sort_values(by=['요청시간'])
    df['요청일자'] = df['요청시간'].apply(lambda x: x.date())
    
    df['속도'] = np.nan
    df.loc[df['상태'] == '하차완료', '속력'] = df.loc[df['상태'] == '하차완료', '거리'] / (df.loc[df['상태'] == '하차완료', '하차지도착시간'] - df.loc[df['상태'] == '하차완료', '승차시간']).dt.total_seconds()
    df['속도'] = (df['속도'] - df['속도'].min()) / (df['속도'].max() - df['속도'].min())
    
    
    df['승차정류장id'] = np.nan
    df['하차정류장id'] = np.nan
    
    
    df['승차정류장id'] = df['승차정류장명'].apply(lambda x: int(x.split('(')[-1].split(')')[0]) if x.split('(')[-1].split(')')[0].isdigit() else np.nan)
    df['하차정류장id'] = df['하차정류장명'].apply(lambda x: int(x.split('(')[-1].split(')')[0]) if x.split('(')[-1].split(')')[0].isdigit() else np.nan)
    df.loc[df['승차정류장id'].notnull(), '승차정류장id'] = df.loc[df['승차정류장id'].notnull(), '승차정류장id'].astype(int)
    df.loc[df['하차정류장id'].notnull(), '하차정류장id'] = df.loc[df['하차정류장id'].notnull(), '하차정류장id'].astype(int)

    
    df['승차정류장'] = np.nan
    df['하차정류장'] = np.nan
    df['승차정류장'] = df['승차정류장명'].apply(lambda x: x[:x.rfind('(')].strip())
    df['하차정류장'] = df['하차정류장명'].apply(lambda x: x[:x.rfind('(')].strip())
    
    df['요청식별id'] = 0
    df['누적요청시간'] = 0
    df['누적요청횟수'] = 1
    df.sort_values(by=['사용자id', '요청시간'], inplace=True) 
    df['요청시간차이'] = df['요청시간'].diff().dt.seconds
    df['요청시간차이'] = df['요청시간차이'].fillna(99999)

    
    df = df.reset_index(drop=True)

    
    j = 1
    for i, row in df.iterrows():
        if j % 100000 == 0:
            print(f'진행률 : {j/len(df)*100:.2f}%')
        j += 1
        if i == 0:
            continue
        if df.iloc[i]['전화번호'] == df.iloc[i-1]['전화번호']  and df.iloc[i]['사용자id'] == df.iloc[i-1]['사용자id'] and df.iloc[i]['요청일자'] == df.iloc[i-1]['요청일자'] and df.iloc[i]['승차정류장명'] == df.iloc[i-1]['승차정류장명'] and df.iloc[i]['하차정류장명'] == df.iloc[i-1]['하차정류장명'] and df.iloc[i]['요청시간차이'] < 600:
            df.iloc[i, df.columns.get_loc('요청식별id')] = df.iloc[i-1]['요청식별id']
        else:
            df.iloc[i, df.columns.get_loc('요청식별id')] = i

    df['누적요청시간'] = df['요청시간'] - df.groupby('요청식별id')['요청시간'].transform('min')
    df['누적요청시간'] = df['누적요청시간'].dt.total_seconds()
    df['누적요청횟수'] = df.groupby('요청식별id')['요청식별id'].transform('count')

    df['실수요여부'] = 0
    df.loc[df.groupby('요청식별id')['요청시간'].idxmax(), '실수요여부'] = 1
    
    df['평가점수_총점'] = df['평가점수_빠른이동'] + df['평가점수_안전운전'] + df['평가점수_버스청결도']

    df.sort_values(by=['요청식별id', '요청시간'], inplace=True) 
    return df    

def preprocess(df):    
    ''' 기본적 전처리를 위한 함수  '''
    columns_to_remove = ['전화번호', 'eta산출api', 'eta최종갱신시간','정류장', '정류장_번호', '정류장_방면','탑승권전달여부', 'qr태깅차량', 'qr수신일자','qr_승차지도착시간차', 'qr태깅단말기','요청시간차이']
    columns_to_rename = ['요청일', '취소일','배차확정일']
    new_columns_name = ['요청시간', '취소시간','배차확정시간']
    df = rename_columns(df,  columns_to_rename, new_columns_name)
    df = convert_to_datetime(df)
    df = change_status(df)
    df = create_new_columns(df)
    df = remove_columns(df, columns_to_remove)
    print(len(df[df['요청식별id'] != 0])/len(df))
    return df

df = preprocess(df)


0.9999993936884177


In [6]:
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 54 columns):
 #   Column       Non-Null Count  Dtype          
---  ------       --------------  -----          
 0   순번           10000 non-null  int64          
 1   호출유형         10000 non-null  object         
 2   지역           10000 non-null  object         
 3   배차차량         904 non-null    object         
 4   승무사원명        904 non-null    object         
 5   상태           10000 non-null  object         
 6   요청시간         10000 non-null  datetime64[ns] 
 7   취소시간         9332 non-null   datetime64[ns] 
 8   승차정류장명       10000 non-null  object         
 9   승차지시뮬레이션시간   904 non-null    datetime64[ns] 
 10  승차지최초도착예정시간  488 non-null    datetime64[ns] 
 11  승차지도착예정시간    901 non-null    datetime64[ns] 
 12  승차지도착시간      693 non-null    datetime64[ns] 
 13  승차시간         668 non-null    datetime64[ns] 
 14  하차정류장명       10000 non-null  object         
 15  하차지시뮬레이션시간   904 non-null    datetime

In [34]:
# save df to csv
df.to_csv('preprocessed.csv', index=False)

실제 이용수요 파악을 위해 동일 사용자가 동일 통행으로 요청을 할 때 그 요청의 간격이 600초 이내인 경우 동일한 요청식별id를 부여 하여,
실제 이용수요를 파악하였고, 각각의 동일하다고 파악된 요청 별로 중복된 요청이 얼마나 있었는지 통계량을 내어보자


81번이나 중복된 요청을 보낸 저분은 과연 버스를 잡았을까?

인천글로벌캠퍼스 다니시는 분인 것 같은데 18분 동안 81회의 요청을 보내 결국 버스 잘 타셨다. 